In [7]:
from pykdtree.kdtree import KDTree
import time
import trimesh
import numpy as np

def filter_mesh_significant_outside_pieces(unfiltered_mesh,main_mesh,
                                           significance_threshold=2000,
                                           n_sample_points=3,
                                          print_flag=False):
    """
    Purpose; will take in a full, unfiltered mesh and find the biggest mesh piece, and then return a list of that mesh 
    with all of the other mesh fragments that are both above the significance_threshold AND outside of the biggest mesh piece

    Pseudocode: 
    1) split the meshes to unconnected pieces
    2) Filter the meshes for only those above the significance_threshold
    3) find the biggest mesh piece
    4) Iterate through all of the remaining pieces:
        a. Determine if mesh inside or outside main mesh
        b. If outside add to final list to return

    Returns: 
    1) list of significant mesh pieces that are not inside of main mesh

    """
    if print_flag:
        print("------Starting the mesh filter for significant outside pieces-------")

    mesh_pieces = unfiltered_mesh.split(only_watertight=False)
    
    if print_flag:
        print(f"There were {len(mesh_pieces)} pieces after mesh split")

    significant_pieces = [m for m in mesh_pieces if len(m.faces) > significance_threshold]

    if print_flag:
        print(f"There were {len(significant_pieces)} pieces found after size threshold")
    if len(significant_pieces) <=0:
        print("THERE WERE NO MESH PIECES GREATER THAN THE significance_threshold")
        return []



    final_mesh_pieces = []

    #final_mesh_pieces.append(main_mesh)
    for i,mesh in enumerate(significant_pieces):
        #get a random sample of points
        # points = np.array(mesh.vertices[:n_sample_points,:]) # OLD WAY OF DOING THIS
        idx = np.random.randint(len(mesh.vertices), size=n_sample_points)
        points = mesh.vertices[idx,:]


        start_time = time.time()
        signed_distance = trimesh.proximity.signed_distance(main_mesh,points)
        #print(f"Total time = {time.time() - start_time}")

        outside_percentage = sum(signed_distance <= 0)/n_sample_points
        if outside_percentage > 0.9:
            final_mesh_pieces.append(mesh)
            #print(f"Mesh piece {i} OUTSIDE mesh")
        else:
            #print(f"Mesh piece {i} inside mesh :( ")
            pass
                
    return final_mesh_pieces
    
def neuron_boolean_difference(main_mesh_verts,
                             main_mesh_faces,
                             child_mesh_verts,
                             child_mesh_faces,
                             distance_threshold=5,
                             significance_threshold=1000,
                             n_sample_points=3,
                             print_flag=False):
    """
    returns the boolean difference of two meshes passed. Only returns meshes pieces that 
    are greater than the size threshold and outside of the main mesh
    
    Function operation: child_mesh - main_mesh
  

    Parameters: 
    main_mesh_verts (np.array): array of the reference mesh vertices
    main_mesh_faces (np.array): array of the reference mesh faces
    child_mesh_verts (np.array): array of the child mesh vertices that is to be compared to the main mesh
    child_mesh_faces (np.array): array of the child mesh faces that is to be compared to the main mesh
    
    Optional parameters:
    -- for controlling the mesh returned --
    distance_threshold (int) : distance away from the reference mesh that vertices from the child
                                mesh will considered distinct and part of the difference mesh 
                                (default=5)
    significance_threshold (int) : number of faces necessary for any distinct/unconnected part of the 
                                    difference mesh to be considered relevant and included in the difference mesh
                                (default=1000)
    n_sample_points (int) : number of vertices to check to see if submesh is a mesh located inside of the main mesh.
                            The less there are the quicker the speed for filtering the difference mesh
                            (default=3)
    
    
    Returns:
    difference_mesh_verts (np.array): array of vertices from the mesh boolean operation of child - main mesh
    difference_mesh_faces (np.array): array of faces from the mesh boolean operation of child - main mesh
    """
    
    #Create the kdtree from main mesh and run the queries
    
    import time
    global_time = time.time()
    start_time = time.time()
    mesh_tree = KDTree(main_mesh_verts)
    distances,closest_node = mesh_tree.query(child_mesh_verts)
    if print_flag:
        print(f"Total time for KDTree creation and queries: {time.time() - start_time}")
    
    if print_flag:
        print("Original number vertices in child mesh = " + str(len(child_mesh_verts)))
    vertex_indices = np.where(distances > distance_threshold)[0]
    if print_flag:
        print("Number of vertices after distance threshold applied =  " + str(len(vertex_indices)))
    
    start_time = time.time()
    set_vertex_indices = set(list(vertex_indices))
    face_indices_lookup = np.linspace(0,len(child_mesh_faces)-1,len(child_mesh_faces)).astype('int')
    face_indices_lookup_bool = [len(set_vertex_indices.intersection(set(tri))) > 0 for tri in child_mesh_faces]
    face_indices_lookup = face_indices_lookup[face_indices_lookup_bool]

    if print_flag:
        print(f"Total time for finding faces after distance threshold applied: {time.time() - start_time}")
    if len(child_mesh_verts)<=0 or len(child_mesh_faces)<=0 or len(face_indices_lookup)<= 0:
        print("inside boolean function and returning because child faces are 0")
        return []
    
    start_time = time.time()
    trimesh_original = trimesh.Trimesh(child_mesh_verts,child_mesh_faces,process=False) 
    new_submesh = trimesh_original.submesh([face_indices_lookup],only_watertight=False,append=True)
    
    pymesh_mesh = trimesh.Trimesh(main_mesh_verts,main_mesh_faces)
    #return new_submesh
    #filter the mesh for only significant pieces on the outside
    returned_mesh = filter_mesh_significant_outside_pieces(new_submesh,pymesh_mesh,
                                                           significance_threshold,
                                                           n_sample_points=n_sample_points,
                                                          print_flag=print_flag)
    
    return returned_mesh
    
    total_returned_mesh = trimesh.Trimesh(vertices = np.array([]),
                                     faces = np.array([]))
    for r in returned_mesh:
        total_returned_mesh = total_returned_mesh + r

    if print_flag:
        print(f"Total time for filtering: {time.time() - start_time}")
    
    if print_flag:
        print(f"Total time for boolean difference: {time.time() - global_time}")
    return total_returned_mesh.vertices,total_returned_mesh.faces

    
    

# Example of Neuron Boolean Difference Parameters that work

In [2]:
#largest_mesh = trimesh.load_mesh("temp/96631955273149705_decimated_poisson_largest_piece.off")
original_mesh = trimesh.load_mesh("temp/107816118160698192_original_decimated.off")

In [3]:
# mesh_pieces =neuron_boolean_difference( largest_mesh.vertices,
#                                                largest_mesh.faces,
#                                                 original_mesh.vertices,
#                                                original_mesh.faces,
                                       
#                                                distance_threshold=1000,
#                                                significance_threshold=1000,
#                                                n_sample_points=10,
#                                       print_flag = False)

# significant_leftovers = trimesh.Trimesh(vertices = mesh_pieces[0],
#                                         faces = mesh_pieces[1]
#                                        )


# Other Helper Functions

In [4]:
import os, contextlib
import pathlib
import subprocess

def run_meshlab_script(mlx_script,input_mesh_file,output_mesh_file):
    script_command = (" -i " + str(input_mesh_file) + " -o " + 
                                    str(output_mesh_file) + " -s " + str(mlx_script))
    #return script_command
    command_to_run = 'xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@ ' + script_command
    #command_to_run = 'meshlabserver ' + script_command
    
    print(command_to_run)
    subprocess_result = subprocess.run(command_to_run,shell=True)
    
    return subprocess_result

def meshlab_fix_manifold_path_specific_mls(input_path_and_filename,
                                           output_path_and_filename="",
                                           segment_id=-1,meshlab_script=""):
    #fix the path if it comes with the extension
    if input_path_and_filename[-4:] == ".off":
        path_and_filename = input_path_and_filename[:-4]
        input_mesh = input_path_and_filename
    else:
        raise Exception("Not passed off file")
    
    
    if output_path_and_filename == "":
        output_mesh = path_and_filename+"_mls.off"
    else:
        output_mesh = output_path_and_filename
    
    if meshlab_script == "":
        meshlab_script = str(pathlib.Path.cwd()) + "/" + "remeshing_remove_non_man_edges.mls"
    
    #print("meshlab_script = " + str(meshlab_script))
    #print("starting meshlabserver fixing non-manifolds")
    subprocess_result_1 = run_meshlab_script(meshlab_script,
                      input_mesh,
                      output_mesh)
    #print("Poisson subprocess_result= "+ str(subprocess_result_1))
    
    if str(subprocess_result_1)[-13:] != "returncode=0)":
        raise Exception('neuron' + str(segment_id) + 
                         ' did not fix the manifold edges')
    
    return output_mesh

In [5]:
#create the output file
##write the OFF file for the neuron
import pathlib
def write_Whole_Neuron_Off_file(vertices=[], 
                                triangles=[],
                                neuron_ID="None",
                                folder="pymesh_NEURONS",
                               path_and_filename="-1"):
    #primary_key = dict(segmentation=1, segment_id=segment_id, decimation_ratio=0.35)
    #vertices, triangles = (mesh_Table_35 & primary_key).fetch1('vertices', 'triangles')
    
    num_vertices = (len(vertices))
    num_faces = len(triangles)
    if path_and_filename == "-1":
        #get the current file location
        file_loc = pathlib.Path.cwd() / folder
        filename = "neuron_" + str(neuron_ID)
        path_and_filename = file_loc / filename
    
    #print("path_and_filename = " + str(path_and_filename))
    
    #open the file and start writing to it    
    f = open(str(path_and_filename) + ".off", "w")
    f.write("OFF\n")
    f.write(str(num_vertices) + " " + str(num_faces) + " 0\n" )
    
    
    #iterate through and write all of the vertices in the file
    for verts in vertices:
        f.write(str(verts[0]) + " " + str(verts[1]) + " " + str(verts[2])+"\n")
    
    #print("Done writing verts")
        
    for faces in triangles:
        f.write("3 " + str(faces[0]) + " " + str(faces[1]) + " " + str(faces[2])+"\n")
    
    #print("Done writing OFF file")
    #f.write("end")
    
    return str(path_and_filename)#,str(filename),str(file_loc)

# Whole Pipeline for One neuron

In [ ]:
"""
Pseudocode For whole run: 

1) Do Poisson Surface Reconstruction 
2) Filter away for largest_poisson_piece:
3) **** Do a check that the reconstruction wasn’t too far away from mesh points on the original mesh ******
4) If not of a significant size then return (Add the largest_poisson_piece to the Surface_Reconstruction_list and say that it was returned)
5) If significant size (add largest_poisson_piece to the Surface_Reconstruction_list)
6) Run skeletonization on it:
    If error then return
7) Subtract the largest_poisson_piece from leftover
8) Find still leftover pieces:
    Divide leftover into separate pieces
    Only keep the significant pieces
9) If no significant leftover pieces
    Return
    Write to log
10) For Each of these significant pieces send to skeleton function (so recursive function)
    Write to log which pieces sent


"""

In [8]:
distance_threshold=1000
significance_threshold=1000
n_sample_points=10
print_flag = False
name2 = "107816118160698192"

import calcification_Module as cm
import time


original_mesh = trimesh.load_mesh("temp/107816118160698192_original_decimated.off")

# create a folder to store the skeletons of that neuron
import os
directory = "./" + name2
if not os.path.exists(directory):
    os.makedirs(directory)
    

current_base_path = str(pathlib.Path.cwd()) + "/"
mesh_base_path = current_base_path + str(name2) + "/"
folder_name = str(name2)

current_layer = 1

def recursive_skeletonizing(verts,faces,current_name,
                           boolean_distance_threshold=1000,
                            boolean_significance_threshold=1000,
                            boolean_n_sample_points=10,
                            boolean_print_flag = False,
                           largest_mesh_significant=3000):
    
    print(f"\n----------Starting {current_name}----------")
    global_time = time.time()
    #print out the off file
    initial_output_path = mesh_base_path + current_name
    
    write_Whole_Neuron_Off_file(vertices=verts, 
                                triangles=faces,
                               path_and_filename=initial_output_path)
    
    
    # do the poisson surface reconstruction
    
    script_name = "poisson_working_meshlab.mls"
    
    input_file_base = initial_output_path
    output_file = input_file_base + "_poisson"
    meshlab_script_path_and_name =current_base_path + script_name

    start_time = time.time()
    print("     Starting Screened Poisson")
    meshlab_fix_manifold_path_specific_mls(input_path_and_filename=input_file_base + ".off",
                                                       output_path_and_filename=output_file + ".off",
                                                     meshlab_script=meshlab_script_path_and_name)
    print(f"     Total_time for Screended Poisson = {time.time() - start_time}")
    
    #2) Filter away for largest_poisson_piece:
    
    new_mesh = trimesh.load_mesh(output_file + ".off")
    mesh_splits = new_mesh.split(only_watertight=True)
    mesh_lengths = np.array([len(split.faces) for split in mesh_splits])
    largest_index = np.where(mesh_lengths == np.max(mesh_lengths))
    largest_mesh = mesh_splits[largest_index][0]
    
    print("len(largest_mesh.vertices) = " + str(len(largest_mesh.vertices)))
    #4) If not of a significant size then return (Add the largest_poisson_piece to the Surface_Reconstruction_list and say that it was returned)
    if len(largest_mesh.vertices) < largest_mesh_significant:
        print(current_name + " returned because largest mesh not significant")
        return
    
    #5) If significant size output the mesh
    largest_mesh_path = mesh_base_path + current_name + "-0"
    write_Whole_Neuron_Off_file(vertices=largest_mesh.vertices, 
                                triangles=largest_mesh.faces,
                               path_and_filename=largest_mesh_path)
    
    
    #6) Run skeletonization on it:
    start_time = time.time()
    print("     Starting Calcification")
    cm.calcification(largest_mesh_path)
    print(f"     Total_time for Calcification = {time.time() - start_time}")
    
    
    #7) Subtract the largest_poisson_piece from leftover
    start_time = time.time()
    print("     Starting Mesh boolean difference")
    mesh_pieces =neuron_boolean_difference( largest_mesh.vertices,
                                               largest_mesh.faces,
                                                verts,
                                               faces,
                                               distance_threshold=boolean_distance_threshold,
                                               significance_threshold=boolean_significance_threshold,
                                               n_sample_points=boolean_n_sample_points,
                                              print_flag = boolean_print_flag)
    print(f"     Total_time for Mesh boolean difference = {time.time() - start_time}")
    
    print(f"Total time for one mesh piece skeleton = {time.time() - global_time}")

    print(f"{current_name} there were {len(mesh_pieces)} significant pieces leftover after largest mesh")
    
    if len(mesh_pieces) <= 0:
        print(f"{current_name} returning because 0 significant pieces")
        return 
    
    for j,piece in enumerate(mesh_pieces):
        new_current_name = current_name + "-" + str(j)
        recursive_skeletonizing(verts = piece.vertices,faces=piece.faces,current_name=new_current_name)
    
    return

    
recursive_skeletonizing(verts = original_mesh.vertices,faces=original_mesh.faces,current_name=name2)


----------Starting 107816118160698192----------
     Starting Screened Poisson
xvfb-run -a -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/Decimation/107816118160698192/107816118160698192.off -o /notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/Decimation/107816118160698192/107816118160698192_poisson.off -s /notebooks/Users/celii/Documents/Complete_Pinky100_Pipeline/notebooks/Platinum_Blender/Decimation/poisson_working_meshlab.mls
     Total_time for Screended Poisson = 48.374598026275635
len(largest_mesh.vertices) = 170930
     Starting Calcification
     Total_time for Calcification = 11.444098711013794
     Starting Mesh boolean difference
     Total_time for Mesh boolean difference = 17.646528959274292
Total time for one mesh piece skeleton = 88.62549066543579
107816118160698192 there were 27 significant pieces leftover after largest mesh

----------Starting 1

# Assemble all of the skeleton files into one

In [9]:
#read in the skeleton files into an array
def read_skeleton_revised(file_path):
    with open(file_path) as f:
        bones = np.array([])
        for line in f.readlines():
            #print(line)
            line = (np.array(line.split()[1:], float).reshape(-1, 3))
            #print(line[:-1])
            #print(line[1:])

            #print(bones.size)
            if bones.size <= 0:
                bones = np.stack((line[:-1],line[1:]),axis=1)
            else:
                bones = np.vstack((bones,(np.stack((line[:-1],line[1:]),axis=1))))
            #print(bones)


    return np.array(bones).astype(float)

In [10]:
cgal_skeleton_file_list = []
import os
for file in os.listdir(name2):
    if file.endswith(".cgal"):
        cgal_skeleton_file_list.append(str(os.path.join(name2, file)))
cgal_skeleton_file_list

bone_array_total = np.vstack([read_skeleton_revised(k) 
                              for k in cgal_skeleton_file_list])

unique_skeleton_verts = bone_array_total.reshape(-1,3)
edges = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)


In [11]:
from Skeleton_Stitcher import stitch_skeleton_with_degree_check, find_skeleton_distance
total_edges = np.array([])

for k in cgal_skeleton_file_list:
    bone_array = read_skeleton_revised(k) 

    #add the skeleton edges to the total edges
    if not total_edges.any():
        total_edges = bone_array
    else:
        total_edges = np.vstack([total_edges,bone_array])


total_edges_stitched = stitch_skeleton_with_degree_check(total_edges)


len_subgraphs AT BEGINNING = 11
min_dist = 1536.5575160077804
min_dist_subgraph_index = 6
min_dist_edge_index = [1994, 1967]
min_dist_edge = [array([1271530.,  711553.,  954589.]), array([1270100.,  712100.,  954459.])]
len_subgraphs AT END= 10
min_dist = 2904.932701457987
min_dist_subgraph_index = 9
min_dist_edge_index = [3824, 3842]
min_dist_edge = [array([1349290.,  625376.,  924148.]), array([1349850.,  624201.,  926745.])]
len_subgraphs AT END= 9
min_dist = 2984.598465455613
min_dist_subgraph_index = 2
min_dist_edge_index = [1289, 1244]
min_dist_edge = [array([1256100.,  658549.,  925690.]), array([1254140.,  657131.,  927438.])]
len_subgraphs AT END= 8
min_dist = 3127.18147858419
min_dist_subgraph_index = 7
min_dist_edge_index = [3166, 3189]
min_dist_edge = [array([1318060.,  731534.,  810315.]), array([1319300.,  734142.,  809115.])]
len_subgraphs AT END= 7
min_dist = 3433.738487421545
min_dist_subgraph_index = 6
min_dist_edge_index = [2682, 2702]
min_dist_edge = [array([1292210

In [12]:
unique_skeleton_verts = total_edges_stitched.reshape(-1,3)
edges = np.arange(0,len(unique_skeleton_verts)).astype("int").reshape(-1,2)

In [13]:
import ipyvolume as ipv
ipv.figure()
# we draw the tetrahedron
original_mesh = trimesh.load_mesh("temp/107816118160698192_original_decimated.off")


mesh = ipv.plot_trisurf(original_mesh.vertices[:,0], 
                        original_mesh.vertices[:,1],
                        original_mesh.vertices[:,2], 
                        triangles=original_mesh.faces, color='orange')

mesh.color = [0., 1., 0., 0.5]
mesh.material.transparent = True
# and also mark the vertices
mesh2 = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
                        unique_skeleton_verts[:,1], 
                        unique_skeleton_verts[:,2], 
                        lines=edges, color='blue')

volume_maxs = np.max(original_mesh.vertices,axis=0)
volume_mins = np.min(original_mesh.vertices,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


# Saving Data for offline visualization

In [14]:
np.savez("107816118160698192_neuron_2_visualizations.npz",
        original_mesh_verts = original_mesh.vertices,
        original_mesh_faces = original_mesh.faces,
        unique_skeleton_verts = unique_skeleton_verts,
        edges=edges)

# Offline Visualization

In [1]:
import ipyvolume as ipv
import numpy as np
import trimesh
ipv.figure()
# we draw the tetrahedron
neuron_data = np.load("107816118160698192_neuron_2_visualizations.npz")
original_mesh_verts = neuron_data["original_mesh_verts"]
original_mesh_faces = neuron_data["original_mesh_faces"]
edges = neuron_data["edges"]
unique_skeleton_verts = neuron_data["unique_skeleton_verts"]


original_mesh = trimesh.Trimesh(vertices=original_mesh_verts,faces=original_mesh_faces)


mesh = ipv.plot_trisurf(original_mesh.vertices[:,0], 
                        original_mesh.vertices[:,1],
                        original_mesh.vertices[:,2], 
                        triangles=original_mesh.faces, color='orange')

mesh.color = [0., 1., 0., 0.5]
mesh.material.transparent = True
# and also mark the vertices
mesh2 = ipv.plot_trisurf(unique_skeleton_verts[:,0], 
                        unique_skeleton_verts[:,1], 
                        unique_skeleton_verts[:,2], 
                        lines=edges, color='blue')

volume_maxs = np.max(original_mesh.vertices,axis=0)
volume_mins = np.min(original_mesh.vertices,axis=0)
ranges = volume_maxs - volume_mins
index = [0,1,2]
max_index = np.argmax(ranges)
min_limits = [0,0,0]
max_limits = [0,0,0]

buffer = 10000
for i in index:
    if i == max_index:
        min_limits[i] = volume_mins[i] - buffer
        max_limits[i] = volume_maxs[i] + buffer 
        continue
    else:
        difference = ranges[max_index] - ranges[i]
        min_limits[i] = volume_mins[i] - difference/2  - buffer
        max_limits[i] = volume_maxs[i] + difference/2 + buffer

#ipv.xyzlim(-2, 2)
ipv.xlim(min_limits[0],max_limits[0])
ipv.ylim(min_limits[1],max_limits[1])
ipv.zlim(min_limits[2],max_limits[2])
ipv.show()

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))
